In [1]:
import numpy as np

# MDP 参数
GRID_SIZE = 5
ACTIONS = ['↑', '↓', '←', '→']  # 上、下、左、右
DISCOUNT_FACTOR = 1.0
THRESHOLD = 1e-4
GOAL_STATE = (1, 3)

def transition(state, action):
    i, j = state
    if action == '↑':
        return max(i - 1, 0), j
    elif action == '↓':
        return min(i + 1, GRID_SIZE - 1), j
    elif action == '←':
        return i, max(j - 1, 0)
    elif action == '→':
        return i, min(j + 1, GRID_SIZE - 1)

def reward_function(state):
    if state == GOAL_STATE:
        return 0  # 目标状态
    return -1  # 其他状态的奖励

def policy_evaluation(policy, V):
    # 初始化迭代次数
    iteration = 0
    while True:
        delta = 0
        new_V = np.zeros_like(V)
        for i in range(GRID_SIZE):
            for j in range(GRID_SIZE):
                state = (i, j)
                if state == GOAL_STATE:  # 跳过目标状态
                    continue
                action_values = []
                for action in policy[state]:
                    next_state = transition(state, action)
                    reward = reward_function(next_state)
                    action_values.append(reward + DISCOUNT_FACTOR * V[next_state])
                new_V[state] = np.mean(action_values)  # 策略下的期望值，如果改为 max 则为贪婪策略，相当于值迭代
                delta = max(delta, abs(new_V[state] - V[state]))
        V = new_V
        
        if delta < THRESHOLD:
            break
        iteration += 1
    return V, iteration

def policy_iteration():
    # 初始化策略和价值函数
    policy = np.empty((GRID_SIZE, GRID_SIZE), dtype=object) # 创建一个 GRID_SIZE x GRID_SIZE 的数组，每个元素都是 ACTIONS 的副本
    for i in range(GRID_SIZE):
        for j in range(GRID_SIZE):
            policy[i, j] = ACTIONS  # 其他状态
    V = np.zeros((GRID_SIZE, GRID_SIZE))  # 初始化价值函数

    iterationDict = {}
    iteration = 0
    while True:
        # 策略评估
        V, eval_iteration = policy_evaluation(policy, V)
        if eval_iteration == 0: # 如果策略评估没有进行任何迭代，说明策略已经收敛，没必要再进行策略改进
            break
        iterationDict[iteration] = eval_iteration

        # 策略改进
        policy_stable = True
        for i in range(GRID_SIZE):
            for j in range(GRID_SIZE):
                state = (i, j)
                if state == GOAL_STATE:  # 跳过目标状态
                    policy[state] = ['G']
                    continue
                action_values = {}
                for action in ACTIONS:
                    next_state = transition(state, action)
                    reward = reward_function(next_state)
                    action_values[action] = reward + DISCOUNT_FACTOR * V[next_state]

                best_actions = [action for action, value in action_values.items() if value == max(action_values.values())]

                if set(policy[state]) != set(best_actions):
                    policy_stable = False
                    policy[state] = best_actions

        if policy_stable:
            break
        iteration += 1

    return policy, V, iterationDict

# 运行策略迭代
optimal_policy, optimal_value, iterationDict = policy_iteration()

# 打印迭代次数
print("Number of Iterations:")
print(iterationDict)
# 打印最优价值函数
print("Optimal Value Function:")
print(optimal_value)
# 打印最优策略
print("Optimal Policy:")
for row in optimal_policy:
    print(' '.join([''.join(a) for a in row]))


Number of Iterations:
{0: 424, 1: 6}
Optimal Value Function:
[[-3. -2. -1.  0. -1.]
 [-2. -1.  0.  0.  0.]
 [-3. -2. -1.  0. -1.]
 [-4. -3. -2. -1. -2.]
 [-5. -4. -3. -2. -3.]]
Optimal Policy:
↓→ ↓→ ↓→ ↓ ↓←
→ → → G ←
↑→ ↑→ ↑→ ↑ ↑←
↑→ ↑→ ↑→ ↑ ↑←
↑→ ↑→ ↑→ ↑ ↑←


In [2]:
DISCOUNT_FACTOR = 1.0
def policy_evaluation(policy, V):
    # 初始化迭代次数
    iteration = 0
    while True:
        delta = 0
        new_V = np.zeros_like(V)
        for i in range(GRID_SIZE):
            for j in range(GRID_SIZE):
                state = (i, j)
                if state == GOAL_STATE:  # 跳过目标状态
                    continue
                value = 0
                for action, action_prob in policy[state].items():
                    next_state = transition(state, action)
                    reward = reward_function(next_state)
                    value += action_prob * (reward + DISCOUNT_FACTOR * V[next_state])
                new_V[state] = value
                delta = max(delta, abs(new_V[state] - V[state]))
        V = new_V
        if delta < THRESHOLD:
            break
        iteration += 1
    return V, iteration

def policy_iteration():
    # 初始随机策略，每个动作概率均等
    policy = {
        (i, j): {a: 1 / len(ACTIONS) for a in ACTIONS}
        for i in range(GRID_SIZE)
        for j in range(GRID_SIZE)
    }
    V = np.zeros((GRID_SIZE, GRID_SIZE))

    iterationDict = {}
    iteration = 0
    while True:
        # 策略评估
        V, eval_iteration = policy_evaluation(policy, V)
        if eval_iteration == 0: # 如果策略评估没有进行任何迭代，说明策略已经收敛，没必要再进行策略改进
            break
        iterationDict[iteration] = eval_iteration

        # 策略改进
        policy_stable = True
        for i in range(GRID_SIZE):
            for j in range(GRID_SIZE):
                state = (i, j)
                if state == GOAL_STATE:
                    continue

                action_values = {}
                for action in ACTIONS:
                    next_state = transition(state, action)
                    reward = reward_function(next_state)
                    action_values[action] = reward + DISCOUNT_FACTOR * V[next_state]

                max_value = max(action_values.values())
                best_actions = [a for a in ACTIONS if action_values[a] == max_value]

                # 更新策略：将最优动作平均分配概率
                new_action_probs = {a: (1 / len(best_actions) if a in best_actions else 0) for a in ACTIONS}

                if policy[state] != new_action_probs:
                    policy_stable = False
                policy[state] = new_action_probs

        if policy_stable:
            break
        iteration += 1

    return policy, V, iterationDict

# 运行策略迭代
optimal_policy, optimal_value, iterationDict = policy_iteration()

# 打印迭代次数
print("Number of Iterations:")
print(iterationDict)
# 打印最优价值函数
print("Optimal Value Function:")
print(optimal_value)
# 打印最优随机策略
print("Optimal Policy:")
for i in range(GRID_SIZE):
    for j in range(GRID_SIZE):
        state = (i, j)
        if state == GOAL_STATE:
            print("G", end="\t")
        else:
            actions = optimal_policy[state]
            print(",".join([f"{a}:{p:.2f}" for a, p in actions.items() if p > 0]), end="\t")
    print()

Number of Iterations:
{0: 424, 1: 6}
Optimal Value Function:
[[-3. -2. -1.  0. -1.]
 [-2. -1.  0.  0.  0.]
 [-3. -2. -1.  0. -1.]
 [-4. -3. -2. -1. -2.]
 [-5. -4. -3. -2. -3.]]
Optimal Policy:
↓:0.50,→:0.50	↓:0.50,→:0.50	↓:0.50,→:0.50	↓:1.00	↓:0.50,←:0.50	
→:1.00	→:1.00	→:1.00	G	←:1.00	
↑:0.50,→:0.50	↑:0.50,→:0.50	↑:0.50,→:0.50	↑:1.00	↑:0.50,←:0.50	
↑:0.50,→:0.50	↑:0.50,→:0.50	↑:0.50,→:0.50	↑:1.00	↑:0.50,←:0.50	
↑:0.50,→:0.50	↑:0.50,→:0.50	↑:0.50,→:0.50	↑:1.00	↑:0.50,←:0.50	


In [3]:
# 示例代码：广义策略迭代

EVALUATION_STEPS = 3  # 每次策略评估只迭代有限步数

def policy_evaluation(policy, V, steps=EVALUATION_STEPS):
    # 初始化迭代次数
    iteration = 0
    for _ in range(steps):
        delta = 0
        new_V = np.zeros_like(V)
        for i in range(GRID_SIZE):
            for j in range(GRID_SIZE):
                state = (i, j)
                if state == GOAL_STATE:
                    continue
                value = 0
                for action, action_prob in policy[state].items():
                    next_state = transition(state, action)
                    reward = reward_function(next_state)
                    value += action_prob * (reward + DISCOUNT_FACTOR * V[next_state])
                new_V[state] = value
                delta = max(delta, abs(new_V[state] - V[state]))
        V = new_V
        if delta < THRESHOLD:
            break
        iteration += 1
    return V, iteration

def policy_improvement(V):
    policy = {}
    for i in range(GRID_SIZE):
        for j in range(GRID_SIZE):
            state = (i, j)
            if state == GOAL_STATE:
                continue
            action_values = {}
            for action in ACTIONS:
                next_state = transition(state, action)
                reward = reward_function(next_state)
                action_values[action] = reward + DISCOUNT_FACTOR * V[next_state]
            
            max_value = max(action_values.values())
            best_actions = [a for a in ACTIONS if action_values[a] == max_value]
            
            # 将最优动作均分概率
            policy[state] = {a: 1 / len(best_actions) if a in best_actions else 0 for a in ACTIONS}
    return policy

def generalized_policy_iteration():
    # 初始随机策略
    policy = {
        (i, j): {a: 1 / len(ACTIONS) for a in ACTIONS}
        for i in range(GRID_SIZE)
        for j in range(GRID_SIZE)
    }
    V = np.zeros((GRID_SIZE, GRID_SIZE))

    iterationDict = {}
    iteration = 0
    while True:
        # 策略评估：只进行有限次迭代
        V, eval_iteration = policy_evaluation(policy, V)
        if eval_iteration == 0: # 如果策略评估没有进行任何迭代，说明策略已经收敛，没必要再进行策略改进
            break
        iterationDict[iteration] = eval_iteration

        # 策略改进
        new_policy = policy_improvement(V)

        if new_policy == policy:
            break
        policy = new_policy
        iteration += 1

    return policy, V, iterationDict

# 运行广义策略迭代
optimal_policy, optimal_value, iterationDict = generalized_policy_iteration()

# 打印迭代次数
print("Number of Iterations:")
print(iterationDict)
# 打印最优价值函数
print("Optimal Value Function:")
print(optimal_value)
# 打印最优策略
print("Optimal Policy:")
for i in range(GRID_SIZE):
    for j in range(GRID_SIZE):
        state = (i, j)
        if state == GOAL_STATE:
            print("G", end="\t")
        else:
            actions = optimal_policy[state]
            print("".join([f"{a}:{p:.2f}" for a, p in actions.items() if p > 0]), end="\t")
    print()

Number of Iterations:
{0: 3, 1: 3, 2: 3}
Optimal Value Function:
[[-3. -2. -1.  0. -1.]
 [-2. -1.  0.  0.  0.]
 [-3. -2. -1.  0. -1.]
 [-4. -3. -2. -1. -2.]
 [-5. -4. -3. -2. -3.]]
Optimal Policy:
↓:0.50→:0.50	↓:0.50→:0.50	↓:0.50→:0.50	↓:1.00	↓:0.50←:0.50	
→:1.00	→:1.00	→:1.00	G	←:1.00	
↑:0.50→:0.50	↑:0.50→:0.50	↑:0.50→:0.50	↑:1.00	↑:0.50←:0.50	
↑:0.50→:0.50	↑:0.50→:0.50	↑:0.50→:0.50	↑:1.00	↑:0.50←:0.50	
↑:0.50→:0.50	↑:0.50→:0.50	↑:0.50→:0.50	↑:1.00	↑:0.50←:0.50	
